In [ ]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from decimal import Decimal

import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import datasets, metrics, model_selection
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer



In [ ]:
matrix = pd.read_csv("data/counts_960.csv")

print(matrix.shape)
deg_samples = matrix[matrix['RNASE_P'] < 40]
matrix = matrix[matrix['RNASE_P'] >= 100]
matrix.shape

In [ ]:
matrix['tpm_ratio']=matrix[['RNASE_P','E_gene','N1_gene','ORF1a']].sum(axis=1)

for g in ['RNASE_P','E_gene','N1_gene','ORF1a']:
    matrix[g]=matrix[g]/matrix['tpm_ratio']

matrix.head()

In [ ]:
X=matrix[['E_gene','N1_gene','ORF1a']]
print(X.shape)

y=matrix.RNA_sample.values
y.shape

# Real test set - do not run more than once !

In [ ]:
# do not run more than once

X_train, X_test, y_train, y_test = model_selection.train_test_split(X , y, random_state=7,train_size=0.8, stratify =y) 
np.savetxt("data/X_train.csv", X_train, delimiter=",") # upload from folder exsiting files if needed
np.savetxt("data/X_test.csv", X_test, delimiter = ",")
np.savetxt("data/y_train.csv", y_train, delimiter = ",")
np.savetxt("data/y_test.csv", y_test, delimiter= ",")

# Split train into validation set

In [ ]:
# take saved train sets

X = np.loadtxt("data/X_train.csv", delimiter=",")
y = np.loadtxt("data/y_train.csv", delimiter=",")

print(X.shape, y.shape)

In [ ]:
#split into "test" (validation) set and the final train set

X_train, X_test, y_train, y_test = model_selection.train_test_split(X , y, random_state=7,train_size=0.8, stratify = y) 
np.savetxt("data/X_train_val.csv", X_train, delimiter=",") # upload from folder exsiting files if needed
np.savetxt("data/X_test_val.csv", X_test, delimiter = ",")
np.savetxt("data/y_train_val.csv", y_train, delimiter = ",")
np.savetxt("data/y_test_val.csv", y_test, delimiter= ",")

In [ ]:
clf=GradientBoostingClassifier(random_state=7, learning_rate= 0.10, subsample=1.0) #default
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print("Mean AUC: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("AUC: {}".format(scores))

In [ ]:
# Grid search - exhaustive

parameters = {
    "loss":["deviance","exponential"], #default is deviance
    "learning_rate": [0.05, 0.075, 0.1, 0.125],#, 0.15], #default is 0.1
    "min_samples_split": np.linspace(0.1, 0.5, 2, 9), #default is 2
    "min_samples_leaf": np.linspace(0.1, 0.5, 1, 9), #default is 1
    "max_depth":[1,3,5,8], #default is 3
    "max_features":["log2","sqrt", None], #default is none
    "criterion": ["friedman_mse",  "mae"], #default is friedman mse
    "subsample":[0.75, 0.8, 0.85, 0.9, 0.95, 1.0], #default is 1.0
    "n_estimators":[10,50,100,150] #default is 100
    }

#passing the scoring function in the GridSearchCV
clf = GridSearchCV(GradientBoostingClassifier(), parameters,scoring="roc_auc",refit=False,cv=10, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.best_params_)


In [ ]:
# optimized model - choose best parameters as needed
clf=GradientBoostingClassifier(random_state=7)

scores = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print("Mean AUC: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("AUC: {}".format(scores))

In [ ]:
# now fit and test

fixed_model = clf.fit(X_train, y_train)

y_hat= fixed_model.predict(X_test)

print("recall: {}".format(recall_score(y_test, y_hat)))
print("precision: {}".format(precision_score(y_test, y_hat)))
print("Accuracy: {}".format(accuracy_score(y_test, y_hat)))
print("Specificity: {}".format(classification_report(y_test, y_hat, output_dict=True)['0.0']['recall']))

y_hat_325 = (fixed_model.predict_proba(X_test)[:,1] >= 0.325).astype(int)
pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test),pd.DataFrame(y_hat_325)],axis=1).to_csv("predictions_GBM.csv")

metrics.plot_roc_curve(clf, X_test, y_test)  # doctest: +SKIP
plt.show()

In [ ]:
clf_NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, ), random_state=1)
scores_NN = cross_val_score(clf_NN, X_train, y_train, cv=10, scoring="roc_auc")
print("Mean AUC: %0.2f (+/- %0.2f)" % (scores_NN.mean(), scores_NN.std() * 2))
print("AUC: {}".format(scores_NN))
clf_NN.fit(X, y)

In [ ]:
mlp_gs = MLPClassifier(random_state=1, max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(10,), (20,), (100,), (100,50), (50,50), (5,10,30),(30,20,10),(100,50,20)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf_NN = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf_NN.fit(X_train, y_train) # X is train samples and y is the corresponding labels


In [ ]:
print(clf_NN.best_params_)

In [ ]:
# now fit and test

clf_NN = MLPClassifier(random_state=7, activation= 'relu', alpha= 0.0001, hidden_layer_sizes= (100,20), learning_rate= 'constant', solver= 'lbfgs')
clf_NN.fit(X_train, y_train)

y_true, y_pred = y_test , clf_NN.predict(X_test)

print("recall: {}".format(recall_score(y_true, y_pred)))
print("precision: {}".format(precision_score(y_true, y_pred)))
print("Accuracy: {}".format(accuracy_score(y_true, y_pred)))
print("Specificity: {}\n".format(classification_report(y_true, y_pred, output_dict=True)['0.0']['recall']))
pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_pred),pd.DataFrame(y_test)],axis=1).to_csv("predictions.csv")
print('Results on the test set:')
print(classification_report(y_true, y_pred))

metrics.plot_roc_curve(clf_NN, X_test, y_true)  # doctest: +SKIP
plt.show()

In [ ]:
# random forest 

clf_RF=RandomForestClassifier(random_state=7)
scores_RF = cross_val_score(clf_RF, X_train, y_train, cv=10, scoring="roc_auc")
print("Mean AUC: %0.2f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("AUC: {}".format(scores_RF))

In [ ]:
# Grid search for RF

# A sample parameter

RF_parameters = {'bootstrap': [True, False],
 'max_depth': [50, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [1, 2, 5],
 'n_estimators': [10, 50 , 100, 200]}

#passing the scoring function in the GridSearchCV
clf_RF = GridSearchCV(RandomForestClassifier(), RF_parameters,scoring="roc_auc",refit=False,cv=5, n_jobs=-1)

clf_RF.fit(X_train, y_train)



In [ ]:
# optimized RF model
clf_RF=RandomForestClassifier(random_state=7)
scores_RF = cross_val_score(clf_RF, X, y, cv=5, scoring="roc_auc")
print("Mean AUC: %0.2f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("AUC: {}".format(scores_RF))

In [ ]:
# now fit 

fixed_model_RF = clf_RF.fit(X_train, y_train)

y_hat_RF= fixed_model_RF.predict(X_test)
#y_hat_RF = (clf.predict_proba(X_test)[:,1] >= 0.58).astype(int)

print("recall: {}".format(recall_score(y_test, y_hat_RF)))
print("precision: {}".format(precision_score(y_test, y_hat_RF)))
print("Accuracy: {}".format(accuracy_score(y_test, y_hat_RF)))
print("Specificity: {}\n".format(classification_report(y_test, y_hat_RF, output_dict=True)['0.0']['recall']))

metrics.plot_roc_curve(clf_RF, X_test, y_test)  # doctest: +SKIP
plt.show()

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(y_test, clf_NN.predict_proba(X_test)[:,1])
auc = metrics.roc_auc_score(y_test, clf_NN.predict_proba(X_test)[:,1])
plt.plot(fpr,tpr,label="Artificial Neural Network, AUC = "+str(round(auc, 3)))

fpr, tpr, thresh = metrics.roc_curve(y_test, clf.predict_proba(X_test)[:,1])
auc = metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
plt.plot(fpr,tpr,label="Gradient Boosting Classifier, AUC = "+str(round(auc, 3)))

fpr, tpr, thresh = metrics.roc_curve(y_test, clf_RF.predict_proba(X_test)[:,1])
auc = metrics.roc_auc_score(y_test, clf_RF.predict_proba(X_test)[:,1])
plt.plot(fpr,tpr,label="Random Forest, AUC = "+str(round(auc, 3)))


plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.plot([0,1], [0,1], color='gray', linestyle='--')

plt.legend(loc=0)

# Thresholds

In [ ]:
def frange(start, stop, step=1.0):
    ''' "range()" like function which accept float type''' 
    i = start
    while i < stop:
        yield i
        i += step

In [ ]:
threshold_table = pd.DataFrame(columns=['Model', 'threshold', 'Accuracy', 'Precision', 'Sensitivity'])

for th in frange(0.25, 0.6, 0.025):
    y_hat = (clf.predict_proba(X_test)[:,1] >= th).astype(int)
    y_hat_NN = (clf_NN.predict_proba(X_test)[:,1] >= th).astype(int)
    report = classification_report(y_test, y_hat, output_dict=True)
    report_NN = classification_report(y_test, y_hat_NN, output_dict=True)

    threshold_table = threshold_table.append({'Model':'Artificial Neural Network', 
                                        'threshold': th,                           
                                        'Precision':precision_score(y_test, y_hat_NN),
                                        'Accuracy':accuracy_score(y_test, y_hat_NN), 
                                        'Sensitivity':recall_score(y_test, y_hat_NN),
                                        'Specificity': report['0.0']['recall']}, ignore_index=True)

    threshold_table = threshold_table.append({'Model':'Gradient Boosting Classifier', 
                                        'threshold': th,                           
                                        'Precision':precision_score(y_test, y_hat),
                                        'Accuracy':accuracy_score(y_test, y_hat), 
                                        'Sensitivity':recall_score(y_test, y_hat),
                                        'Specificity': report['0.0']['recall']}, ignore_index=True)

threshold_table.set_index('Model').sort_values(['Model','threshold'])

# Save models

In [ ]:
import pickle

filename = 'GBM_model_val_noRNASE.sav'
pickle.dump(clf, open(filename, 'wb'))

filename = 'ANN_model_val_noRNASE.sav'
pickle.dump(clf_NN, open(filename, 'wb'))

filename = 'RF_model_val_noRNASE.sav'
pickle.dump(clf_RF, open(filename, 'wb'))


In [ ]:
import joblib

filename = 'GBM_model_val_rnaseInc.joblib'
joblib.dump(clf, filename)

filename = 'ANN_model_val_rnaseInc.joblib'
joblib.dump(clf_NN, filename)

filename = 'RF_model_val_rnaseInc.joblib'
joblib.dump(clf_RF, filename)
